In [40]:
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as sc
import behavior.boris_extraction as boris
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel
import spike.spike_analysis.spike_recording as rec

In [41]:
def random_event_generator(start, stop, len_event, no_events):
    # Generate random events for baseline period
    total_duration = stop - start
    possible_events = np.arange(int(total_duration / len_event))
    pot_events = np.random.choice(possible_events, size = (no_events), replace = False)
    pot_events = np.sort(pot_events)
    events = []
    for i in pot_events: 
        event_start = (start + (len_event * i)) * 1000
        event_stop = (event_start + (len_event * 1000))
        events.append(np.array([event_start, event_stop]))
    return(np.array(events))

In [57]:
# Prepare the behaviors dictionary
behavior_dfs = {}
path = r"C:\Users\zhaoz\Desktop\Research\data\aggregated_csvs"
for file in os.listdir(path):
	if file.endswith(".csv"):
		behavior_dfs[file] = pd.read_csv(os.path.join(path, file))

behavior_dicts = {}
anogenital_sniffing_dicts = {}
facial_sniffing_dicts = {}
event_timestamp = {}
min_bout = 0.3
min_iti = 0.5
subject = ['subject']
order_dict = {'T': 'novel object', 'B': 'familiar object', 'N': 'novel mouse', 'C': 'familiar mouse'}

for filename, df in behavior_dfs.items():
	#print(filename)
	mouse_in = list(df[df['Behavior'] == 'mice back in']['Start (s)'])
	mouse_out = list(df[df['Behavior'] == 'mice taken out']['Start (s)'])
	if filename == '23_object_TBCN.csv':
		mouse_in.insert(0, 602)
		mouse_out.insert(0, 900)
		mouse_in.insert(1, 902)
		mouse_out.insert(1, 1201)
	if filename == '32_object_NBTC.csv': 
		mouse_in.insert(1, 922)
		mouse_in.insert(2, 1222)
		mouse_out.insert(1, 1222)
		mouse_out.insert(2,1521)
	if filename == '31_object_BNTC.csv':
		mouse_in.insert(0, 608)
		mouse_in.insert(2, 1213)
		mouse_out.insert(0, 906)
		mouse_out.insert(2,1521)
	df_1 = df[df['Start (s)'] < mouse_out[0]]
	df_2 = df[(df['Start (s)'] > mouse_in[1]) &
            (df['Start (s)'] < mouse_out[1])]
	df_3 = df[(df['Start (s)'] > mouse_in[2]) &
            (df['Start (s)'] < mouse_out[2])]
	df_4 = df[(df['Start (s)'] > mouse_in[3])]
	timestamp = [(mouse_in[0]*1000, mouse_out[0]*1000),
			  (mouse_in[1]*1000, mouse_out[1]*1000),
			  (mouse_in[2]*1000, mouse_out[2]*1000),
			  (mouse_in[3]*1000,)]
	print(timestamp)
	# anogenital sniffing
	anogenital_sniffing_arrays = []
	anogenital_sniffing_arrays.append(boris.get_behavior_bouts(df_1,
                                    subject = subject,
                                    behavior = ['anogenital sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	anogenital_sniffing_arrays.append(boris.get_behavior_bouts(df_2,
                                    subject = subject,
                                    behavior = ['anogenital sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	anogenital_sniffing_arrays.append(boris.get_behavior_bouts(df_3,    
                                    subject = subject,
                                    behavior = ['anogenital sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	anogenital_sniffing_arrays.append(boris.get_behavior_bouts(df_4,        
                                    subject = subject,
                                    behavior = ['anogenital sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	# facial sniffing
	facial_sniffing_arrays = []
	facial_sniffing_arrays.append(boris.get_behavior_bouts(df_1,
                                    subject = subject,
                                    behavior = ['facial sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	facial_sniffing_arrays.append(boris.get_behavior_bouts(df_2,
                                    subject = subject,
                                    behavior = ['facial sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	facial_sniffing_arrays.append(boris.get_behavior_bouts(df_3,    
                                    subject = subject,
                                    behavior = ['facial sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	facial_sniffing_arrays.append(boris.get_behavior_bouts(df_4,        
                                    subject = subject,
                                    behavior = ['facial sniffing'],
                                    min_bout = min_bout,
                                    min_iti = min_iti))
	baseline_array = random_event_generator(0, mouse_in[0], 1, 12)
	order = filename.split("_")[-1].split(".")[0]
	behavior_dict = {}
	rec_timestamp = {}
	for i in range(len(order)):
		behavior_dict[f'{order_dict[order[i]]}'] = {'anogenital sniffing': anogenital_sniffing_arrays[i]*1000,
													'facial sniffing': facial_sniffing_arrays[i]*1000}
		if order[i] == 'C' or order[i] == 'N':
			rec_timestamp[f'{order_dict[order[i]]}'] = timestamp[i]
	behavior_dict['baseline'] = baseline_array
	behavior_dicts[filename] = behavior_dict
	event_timestamp[filename] = rec_timestamp

print(behavior_dicts)
print(event_timestamp)

[(601200.0, 910533.0), (915333.0, 1199867.0), (1199933.0, 1504733.0), (1515467.0,)]
[(602000, 900000), (902000, 1201000), (1203600.0, 1505000.0), (1519533.0,)]
[(608000, 906000), (906667.0, 1212200.0), (1213000, 1521000), (1511733.0,)]
[(620333.0, 922067.0), (922000, 1222000), (1222000, 1521000), (1523000.0,)]
[(610705.0, 909851.0), (910601.0, 1213086.0), (1230585.0, 1526238.0), (1529239.0,)]
[(608600.0, 904533.0), (904667.0, 1209333.0), (1221733.0, 1524800.0), (1532467.0,)]
{'22_object_CTBN.csv': {'familiar mouse': {'anogenital sniffing': array([[605400., 606399.],
       [607600., 608866.],
       [609667., 610932.],
       [611867., 612600.],
       [614133., 614532.],
       [616267., 616933.],
       [630933., 632733.],
       [643933., 645066.],
       [678400., 680533.],
       [696000., 696467.],
       [816400., 818267.],
       [884133., 885400.],
       [886400., 886867.],
       [904133., 904600.]]), 'facial sniffing': array([[614533., 615067.],
       [628200., 630932.],
 

In [43]:
novel_event_dict = {}
cagemate_event_dict = {}
subject_dict = {}
j = 1
for recording, event_dict in behavior_dicts.items():
	recording_name_parts = (recording.split(".")[0]).split("_")[:-1]
	recording_name = "_".join(recording_name_parts) + "_merged.rec"
	# Familiar mouse
	novel_event_dict[recording_name] = {}
	novel_event_dict[recording_name]['anogenital sniffing'] = event_dict['novel mouse']['anogenital sniffing']
	novel_event_dict[recording_name]['facial sniffing'] = event_dict['novel mouse']['facial sniffing']
	novel_event_dict[recording_name]['baseline'] = event_dict['baseline']
	# Novel mouse
	cagemate_event_dict[recording_name] = {}
	cagemate_event_dict[recording_name]['anogenital sniffing'] = event_dict['familiar mouse']['anogenital sniffing']
	cagemate_event_dict[recording_name]['facial sniffing'] = event_dict['familiar mouse']['facial sniffing']
	cagemate_event_dict[recording_name]['baseline'] = event_dict['baseline']
	# Assign subject
	subject = f"Subject {j}"
	subject_dict[recording_name] = subject
	j += 1

print(novel_event_dict)
print(cagemate_event_dict)
print(subject_dict)

{'22_object_merged.rec': {'anogenital sniffing': array([[1518267., 1518732.],
       [1520133., 1525199.],
       [1525933., 1526399.],
       [1532667., 1533332.],
       [1538800., 1539266.],
       [1541000., 1543133.],
       [1545000., 1545599.],
       [1548000., 1548466.],
       [1549533., 1549932.],
       [1551467., 1551800.],
       [1559800., 1561200.],
       [1562933., 1563266.],
       [1564267., 1564666.],
       [1578267., 1578932.],
       [1586333., 1588266.],
       [1590467., 1591399.],
       [1597267., 1598399.],
       [1649067., 1649600.],
       [1679533., 1680000.],
       [1685867., 1686200.],
       [1722867., 1723932.],
       [1750600., 1752133.],
       [1754267., 1756199.],
       [1759067., 1759800.],
       [1761333., 1762399.],
       [1767133., 1767666.],
       [1785933., 1786599.],
       [1809200., 1809600.]]), 'facial sniffing': array([[1527400., 1531467.],
       [1558467., 1559799.],
       [1588267., 1589466.],
       [1620533., 1621400.],
  

In [44]:
# Initiate the spike collection object
novel_collection = sc.SpikeCollection(path = r"C:\Users\zhaoz\Desktop\Research\data", event_dict= novel_event_dict, subject_dict= subject_dict, sampling_rate= 20000)
cagemate_collection = sc.SpikeCollection(path = r"C:\Users\zhaoz\Desktop\Research\data", event_dict= cagemate_event_dict, subject_dict= subject_dict, sampling_rate= 20000)

loading  22_object_merged.rec
loading  23_object_merged.rec
23_object_merged.rec has no good units
and will not be included in the collection
loading  31_object_merged.rec
loading  32_object_merged.rec
loading  41_object_merged.rec
loading  44_object_merged.rec
loading  22_object_merged.rec
loading  23_object_merged.rec
23_object_merged.rec has no good units
and will not be included in the collection
loading  31_object_merged.rec
loading  32_object_merged.rec
loading  41_object_merged.rec
loading  44_object_merged.rec


In [45]:
novel_collection.analyze(timebin = 100, ignore_freq=0.5)
cagemate_collection.analyze(timebin = 100, ignore_freq=0.5)
event_length = 3
baseline_window = 1
offset = 0
cagemate_dfs = []
novel_dfs = []
cagemate_event_length = {'anogenital sniffing': 2.8, 'facial sniffing':1.6, 'baseline': 3}
novel_event_length_dict = {'anogenital sniffing': 1.8, 'facial sniffing':1.7, 'baseline': 2}
events = ['anogenital sniffing', 'facial sniffing', 'baseline']

# Create first event's dataframes
cagemate_merged = single_cell.wilcoxon_collection(cagemate_collection, 
                                                  event=events[0], 
												  event_length=event_length, 
												  baseline_window=baseline_window, 
												  offset=offset, 
												  exclude_offset=True, 
												  plot=False)
novel_merged = single_cell.wilcoxon_collection(novel_collection, 
                                               event=events[0], 
											   event_length=event_length, 
											   baseline_window=baseline_window, 
											   offset=offset, 
											   exclude_offset = True, 
											   plot=False)

# Merge remaining events with suffixes
for i, event in enumerate(events[1:], 1):
   suffix = f"_{event}"
   cagemate_df = single_cell.wilcoxon_collection(cagemate_collection, event=event, event_length=cagemate_event_length[event], baseline_window=baseline_window, offset=offset, plot=False)
   novel_df = single_cell.wilcoxon_collection(novel_collection, event=event, event_length=novel_event_length_dict[event], baseline_window=baseline_window, offset=offset, plot=False)
   
   cagemate_merged = pd.merge(cagemate_merged, cagemate_df, on=['Recording', 'Subject', 'original unit id'], suffixes=('', suffix))
   novel_merged = pd.merge(novel_merged, novel_df, on=['Recording', 'Subject', 'original unit id'], suffixes=('', suffix))

All set to analyze
All set to analyze
Wilcoxon can't be done on 31_object_merged.rec anogenital sniffing, because <6 samples
Wilcoxon can't be done on 31_object_merged.rec anogenital sniffing, because <6 samples
Wilcoxon can't be done on 31_object_merged.rec facial sniffing, because <6 samples
Wilcoxon can't be done on 32_object_merged.rec facial sniffing, because <6 samples
Wilcoxon can't be done on 44_object_merged.rec facial sniffing, because <6 samples
Wilcoxon can't be done on 31_object_merged.rec facial sniffing, because <6 samples


In [49]:
for recording in novel_collection.recordings:
	print(len(recording.unit_firing_rate_array))

18106
18303
18309
18303
18095
